<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/intervviewZero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install git+https://github.com/openai/whisper.git
!pip install -U git+https://github.com/jianfch/stable-ts.git

  Cloning https://github.com/jianfch/stable-ts.git to /tmp/pip-req-build-xrq94fxs
  Running command git clone --filter=blob:none --quiet https://github.com/jianfch/stable-ts.git /tmp/pip-req-build-xrq94fxs
  Resolved https://github.com/jianfch/stable-ts.git to commit 7dab1714c5915e3efe070ae697ade1d04f5c8c42
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 45.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.6 MB/s eta 0:00:00
  Created wheel for stable-ts: filename=stable_ts-2.17.5-py3-none-any.whl size=136044 sha256=4116b38492e8da2fbc4f85ff4d6e04ac50fb6f2dc00479b6b42daedd3898a613
  Stored in directory: /tmp/pip-ephem-wheel-cache-4nykvp16/wheels/5a/48/64/a463d57ac05105e1692e3649ca76cea98a8867262d

In [ ]:
!pip install moviepy

In [ ]:
!pip install ffmpeg-python

In [ ]:
#import whisper
import stable_whisper
import ffmpeg
import numpy as np
import json, re, os
import glob
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from moviepy.editor import VideoFileClip
from moviepy.editor import AudioFileClip
from moviepy.editor import CompositeVideoClip

In [ ]:
!cp ./drive/MyDrive/IZ-2218確認４/*.* ./

In [ ]:
def is_webm(path):
    with open('./' + path) as f:
        for _line in f:
            if _line.rstrip()[-5:] == '.webm':
                return True
    return False

In [ ]:
def conv_webm(input_file):
    #!ffmpeg -allowed_extensions ALL -f hls -i ./drive/MyDrive/IZ-2218確認３/ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_907719631__uid_e_video.m3u8 -c:v copy -y ./ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_907719631__uid_e_video.webm
    webm_tmp = "./video_tmp.webm"
    try:
        (
            ffmpeg
            .input('./' + input_file, allowed_extensions='ALL')
            .output(webm_tmp, vcodec="copy")
            .overwrite_output()
            .run(capture_stderr=True, capture_stdout=True)
        )
        _clip = VideoFileClip(webm_tmp)
    except ffmpeg.Error as e:
        #print(f"{e.stderr.decode()}")
        #print(f"標準出力: {e.stdout.decode()}")
        _clip = None
    return _clip

In [ ]:
def get_video_list(path):
    video_path = f'{path}*_video.m3u8'
    video_paths = glob.glob(video_path)
    video_paths.sort()
    return video_paths
def get_audio_list(path):
    audio_path = f'{path}*_audio.m3u8'
    audio_paths = glob.glob(audio_path)
    audio_paths.sort()
    return audio_paths

In [ ]:
def get_speaker_monitor(result, sorted, video_paths):
    prog = re.compile('教えて|聞か|聞き|願い|ください|ですか(?!ね)|でしょうか(?!ね)|ましたか|されましたか|いつ頃')
    character = {}
    INT_COUNT = []
    for c in range(len(result)):
        INT_COUNT.append([0, 0, 0, ''])
    for srt, end, txt, spk in sorted:
        count = len(prog.findall(txt))
        spk_no = int(spk.split('_')[-1])
        INT_COUNT[spk_no][0] += 1
        INT_COUNT[spk_no][3] = spk
        if count > 0:
            INT_COUNT[spk_no][1] += count
    for c, cnt in enumerate(INT_COUNT):
        INT_COUNT[c][2] = cnt[1] / cnt[0]
    INT_COUNT.index(min(INT_COUNT))

    rate = [c[2] for c in INT_COUNT]
    min_index = rate.index(min(rate))

    for i, intv in enumerate(INT_COUNT):
        user_id = audio_paths[i].split('_')[-5]
        if i == min_index:
            character[intv[3]] = {'character': "モニター", 'user_id': user_id}
        else:
            character[intv[3]] = {'character': "インタビューア_" + intv[3].split('_')[-1], 'user_id': user_id}
    return character

In [ ]:
path = './'
video_paths = get_video_list(path)
audio_paths = get_audio_list(path)
interview_vido_name = '_'.join(video_paths[0].split('/')[1].split('_')[0: 2]) + '.mp4'
interview_json_name = '_'.join(video_paths[0].split('/')[1].split('_')[0: 2]) + '.json'

In [ ]:
video = []
for video_path in video_paths:
    _video = conv_webm(video_path) if is_webm(video_path) else VideoFileClip(video_path)
    video.append(_video)

In [ ]:
audio = []
wav_paths = []
for audio_path in audio_paths:
    wav_path = '.'.join(audio_path.split('.')[:-1]) + '.wav'
    audio_tmp = AudioFileClip(audio_path)
    audio_tmp.write_audiofile(wav_path, ffmpeg_params=["-ac", "1"])
    audio.append(audio_tmp)
    wav_paths.append(wav_path)
print(audio)

In [ ]:
model = stable_whisper.load_model('large')
stable_whisper.modify_model(model)

In [ ]:
asr_result = []
for wav_patn in wav_paths:
    print(f'\n---- {wav_patn} ----')
    _result = model.transcribe(wav_patn, verbose=True, fp16=False, language="ja", word_timestamps=True)
    asr_result.append(_result)

In [ ]:
_seg = []
for spk, res in enumerate(asr_result):
    for seg in res.segments:
        _seg.append([seg.start, seg.end, seg.text, f'speaker_{format(spk, "02") }'])

In [ ]:
segments_sorted = sorted(_seg, key=lambda x:(x[0], x[1]))

In [ ]:
characters = get_speaker_monitor(asr_result, segments_sorted, video_paths)
characters

In [ ]:
InterReview_json = {"movie_name": interview_vido_name}
speak_json = []
for start, end, text, speaker in segments_sorted:
    _speak = {"start": f'{start:.2f}s', "end": f'{end:.2f}s', 'speaker': characters[speaker]['character'], 'speak': text}
    speak_json.append(_speak)
InterReview_json['InterReview'] = speak_json

with open('./' + interview_json_name, 'w', encoding='utf-8') as f:
    json.dump(InterReview_json, f, indent=4, ensure_ascii=False)


In [ ]:
video_part = []
cnt = 1
for c, char in enumerate(characters):
    if char['character'] == 'モニタ':
        video_part[0] = video[c]
    else:
        video_part[cnt] = cideo[c]
        cnt += 1

In [ ]:
w,h = moviesize = video_part[0].size
wipe_w = 128
wipe_h = 72
wipe_margin_right = 4
wipe_margin_top = 4
wipe = []
for v in range(len(video_part) - 1):
    _wipe = (video_part[v + 1].
            resize((wipe_w, wipe_h)).
            margin(top=wipe_margin_top, right=wipe_margin_right, opacity=0).
            set_pos((w - wipe_w - wipe_margin_right, wipe_h * 0 + wipe_margin_right * 1)) )
   wipe.append(_wipe)

In [ ]:
combined_clip = CompositeVideoClip([video_part[0], wipe1, wipe2])

In [ ]:
combined_clip.write_videofile(combined_path, fps=30, audio_codec = 'aac')